In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/data_.csv")

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import Dataset

model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
import pandas as pd
from datasets import Dataset

data_dict = test_df.to_dict(orient='list')
dataset = Dataset.from_dict(data_dict)

In [ ]:
def prepare_data(data):

    # print("\n\n\nKeys:", list(data.keys()))
    # for key, value in data.items():
    #     print(".................................................")
    #     print("here is key ---->>>",key)
    #     print("here is value ---->>>",value)
        
    model_inputs = tokenizer(data["input"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(data["output"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    
    # print("\n\n\nKeys:", list(model_inputs.keys()))
    # print("labels:", list(labels.keys()))
    # print(model_inputs["input_ids"])
    # print(model_inputs["attention_mask"])

    return model_inputs
    
tokenized_dataset = dataset.map(prepare_data)

# for row in dataset["content"]:
#     # prepare_data({"content": row})
#     prepare_data(row)
#     break 

In [ ]:
# Remove unnecessary columns
columns_to_remove = ['output', 'input']
tokenized_dataset = tokenized_dataset.remove_columns(columns_to_remove)



# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Set the format of your dataset
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Check a few examples
print(train_dataset[0])
print(tokenizer.decode(train_dataset[0]['input_ids']))
print(tokenizer.decode(train_dataset[0]['labels']))

# # Create a data collator
# from transformers import DataCollatorForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)